# Seq2Seq
實現基礎版的Seq2Seq，輸入一個英文單字，模型將輸出一個對字母排序後的單字
<br>輸入 : hello
<br>輸出 : ehllo

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import time
import copy
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# 讀取數據

In [2]:
f = open('data/letters_source.txt', 'r', encoding = 'utf-8')
source_data = f.read()

f = open('data/letters_target.txt', 'r', encoding = 'utf-8')
target_data = f.read()

# 數據預處理

In [3]:
def extract_character_vocab(data):
    
    special_words = ['<PAD>' , '<UNK>' , '<GO>' , '<EOS>']
    
    words = []
    for line in data.split('\n'):
        for character in line:
            if character not in words:
                words.append(character)

    # 將四個特殊字加入詞庫       
    int_to_vocab = {idx: word for idx , word in enumerate(special_words + words)}
    vocab_to_int = dict(zip(int_to_vocab.values() , int_to_vocab.keys()))

    return int_to_vocab, vocab_to_int

In [4]:
source_int_to_letter , source_letter_to_int = extract_character_vocab(source_data)
target_int_to_letter , target_letter_to_int = extract_character_vocab(target_data)

# 將所有字母轉換成index
source_int = []
for line in source_data.split('\n'):
    temp = []
    for letter in line:
        temp.append(source_letter_to_int[letter])
    source_int.append(temp)    
        
target_int = []
for line in target_data.split('\n'):
    temp = []
    for letter in line:
        temp.append(target_letter_to_int[letter])
    temp = temp + [target_letter_to_int['<EOS>']]
    target_int.append(temp)     

In [5]:
# 超参数
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_hidden_unit = 50
# Number of Layers
num_layers = 1
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001

# Build Model

## 輸入層

In [6]:
input_data = tf.placeholder(tf.int32 , [None , None] , name = 'inputs')
targets = tf.placeholder(tf.int32 , [None , None] , name = 'targets')
lr = tf.placeholder(tf.float32 , name = 'learning_rate')

source_sequence_length = tf.placeholder(tf.int32 , [None ,] , name = 'source_sequence_length')
target_sequence_length = tf.placeholder(tf.int32 , [None ,] , name = 'target_sequence_length')
# 決定target序列最大長度（之後target_sequence_length和source_sequence_length會作為feed_dict的參數）
max_target_sequence_length = tf.reduce_max(target_sequence_length , name = 'max_target_len')

## Encoder

需要對source數據進行embedding，再傳入Decoder中的RNN

In [7]:
# input_data: 輸入tensor
# rnn_hidden_unit: rnn隱層結點數量
# num_layers: rnn cell的層數
# source_sequence_length: source數據的序列長度
# source_vocab_size: source數據的詞庫大小
# encoding_embedding_size: embedding的向量維度

# Encoder embedding
'''
encoder_embed_input = tf.contrib.layers.embed_sequence(input_data , source_vocab_size , encoding_embedding_size) 
                                                  ⇕ 相當於
encoder_embeddings = tf.Variable(tf.random_uniform([source_vocab_size , encoding_embedding_size]))
encoder_embed_input = tf.nn.embedding_lookup(encoder_embeddings , input_data)

若懶得寫兩行程式可以直接用tf.contrib.layers.embed_sequence這個函數
介紹 : https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence
'''
source_vocab_size = len(source_letter_to_int)
encoder_embeddings = tf.Variable(tf.random_uniform([source_vocab_size , encoding_embedding_size]))
encoder_embed_input = tf.nn.embedding_lookup(encoder_embeddings , input_data)


def get_lstm_cell(rnn_hidden_unit):
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_hidden_unit, 
                                        initializer = tf.random_uniform_initializer(-0.1 , 0.1))
    return lstm_cell

cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_hidden_unit) for _ in range(num_layers)])

encoder_output, encoder_state = tf.nn.dynamic_rnn(cell, 
                                                  encoder_embed_input, 
                                                  sequence_length = source_sequence_length,
                                                  dtype = tf.float32)

## Decoder

In [8]:
# 預處理後的decoder輸入
# 在batch中每一筆data最前面加上<GO>，並移除最後一個字，所以每一筆data的詞的數目並無改變

# cut掉最後一個字
# ending = tf.strided_slice(targets , [0, 0] , [batch_size, -1] , [1, 1]) # 等同於 ending = tf.identity(targets[: , 0:-1])
ending = tf.identity(targets[: , 0:-1])
decoder_input = tf.concat([tf.fill([batch_size, 1] , target_letter_to_int['<GO>']) , ending] , axis = 1)

### tf.contrib.seq2seq.TrainingHelper:(Training 階段，還有其他種類的Helper)
### 訓練時採用teacher forcing，永遠把ground truth輸入給模型，不管模型前一步預測結果是否正確
此函數為Decoder端用來訓練的參數，這個函數不會把t-1時刻的輸出當作t時刻的輸入，而是把target中的真實質直接輸入給RNN<br>
主要參數是inputs與sequence_length，返回helper對象，可以做為Basic Decoder函數的參數
<br><br><br>

### tf.contrib.seq2seq.GreedyEmbeddingHelper:(Inference 階段，還有不同sample手段的Helper)
### 它和TrainingHelper的區別在於它會把t-1時刻的輸出經過embedding層作為t時刻的輸入
• greedy decoding：每一次把模型認為機率最大的 token 輸入給下一時刻<br>
• beam search decoding：每次保留 top k 的預測結果，解碼得到（近似） k best 序列 <br>
• sample decoding：每一步從模型預測的機率分布中隨機取樣一個 token 輸入給下一時刻

In [9]:
# decoding_embedding_size: embedding的向量維度
# num_layers: rnn cell的層數
# rnn_size: RNN單元的隱層結點數量
# target_sequence_length: target數據序列長度
# max_target_sequence_length: target數據序列最大長度
# encoder_state: encoder端編碼的狀態向量
# decoder_input: decoder端輸入

# 1. Embedding，需要對target數據進行embedding，再傳入Decoder中的RNN
target_vocab_size = len(target_letter_to_int)
decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size , decoding_embedding_size]))
decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings , decoder_input)

# 2. 建造Decoder中的RNN單元
def get_decoder_cell(rnn_hidden_unit):
    decoder_cell = tf.contrib.rnn.LSTMCell(rnn_hidden_unit,
                                           initializer = tf.random_uniform_initializer(-0.1 , 0.1))
    return decoder_cell
cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_hidden_unit) for _ in range(num_layers)])
 
# 3. Output全連接層
output_layer = Dense(target_vocab_size,
                     kernel_initializer = tf.truncated_normal_initializer(mean = 0.0 , stddev = 0.1))


# 4. Training decoder
with tf.variable_scope('decoder'):
    # tf.contrib.seq2seq.TrainingHelper即是採用Teacher Forcing的方法
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs = decoder_embed_input,
                                                        sequence_length = target_sequence_length,
                                                        time_major = False)
    # 建造decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 
    
    # decoder_output包含 rnn_output 與 sample_id
    # rnn_output: [batch_size, decoder_targets_length, vocab_size]，保存decode每個時刻每個單詞的概率，可以用來計算loss
    # sample_id: [batch_size], tf.int32，保存最終的編碼結果。可以表示最後的答案
    training_decoder_output ,\
    training_final_state ,\
    training_final_sequence_lengths =\
    tf.contrib.seq2seq.dynamic_decode(training_decoder,                                          
                                      impute_finished = True,
                                      maximum_iterations = max_target_sequence_length)
     
    
     
# 5. Predicting decoder 與 Training decoder共享參數
with tf.variable_scope('decoder'):
    
    tf.get_variable_scope().reuse_variables() 
    
    # 創建一個常量tensor並覆制為batch_size的大小
    start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32),
                           [batch_size], 
                           name = 'start_tokens')
    
    # GreedyEmbeddingHelper採取argmax抽樣演算法來得到輸出id，並且經過embedding層作為下一時刻的輸入
    predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                 start_tokens,
                                                                 target_letter_to_int['<EOS>'])
    

    predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                         predicting_helper,
                                                         encoder_state,
                                                         output_layer)
    
    predicting_decoder_output ,\
    predicting_final_state ,\
    predicting_final_sequence_lengths =\
    tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                      impute_finished = True,
                                      maximum_iterations = max_target_sequence_length)                                              

In [10]:
training_logits = tf.identity(training_decoder_output.rnn_output ,  name = 'logits')
predicting_logits = tf.identity(predicting_decoder_output.sample_id ,  name = 'predictions')

'''
target_sequence_length : [4 , 2 , 3]

max_target_sequence_length : 8

=> masks的輸出長這樣 : 1 1 1 1 0 0 0 0  (4)
                       1 1 0 0 0 0 0 0  (2)
                       1 1 1 0 0 0 0 0  (3)
-> 0的部分代表是補0的地方，不列入loss的計算，可以加快運算速度
'''                
               
masks = tf.sequence_mask(target_sequence_length , 
                         max_target_sequence_length, 
                         dtype = tf.float32, 
                         name = 'masks')


with tf.variable_scope('optimization'):        
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(training_logits,
                                            targets,
                                            masks)

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Batches

In [11]:
def get_batches(source , target , batch_index , batch_i):         
    source_array = np.array(source)      
    target_array = np.array(target)   
    source_batch = list(source_array[batch_index[batch_i]])
    target_batch = list(target_array[batch_index[batch_i]])
    
    max_sentence_source = max([len(sentence) for sentence in source_batch]) 
    max_sentence_target = max([len(sentence) for sentence in target_batch]) 
    
    # 將長度少於max_sentence的補0，使每個序列的長度都是max_sentence
    pad_source_batch , pad_target_batch = [] , []
    source_length , target_length = [] , []   
    for i in range(0 , batch_size):
        temp_source =  copy.deepcopy(source_batch[i])
        while len(temp_source) < max_sentence_source:
            temp_source.append(0)
        pad_source_batch.append(temp_source)    
        
        temp_target = copy.deepcopy(target_batch[i])
        while len(temp_target) < max_sentence_target:
            temp_target.append(0)
        pad_target_batch.append(temp_target)   
        
        # 記錄每條數據的長度
        source_length.append(len(source_batch[i]))
        target_length.append(len(target_batch[i]))
       
    pad_source_batch = np.array(pad_source_batch)
    pad_target_batch = np.array(pad_target_batch)
    source_length = np.array(source_length)
    target_length = np.array(target_length)
    
    return pad_source_batch , pad_target_batch , source_length , target_length

In [12]:
# 將數據集分割為train和validation
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]
# 留出一個batch進行驗證
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]

## Training

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch_i in range(0 , epochs):
    
    # 在每進行一個epoch前，把每個batch的index先決定出來
    batch_index = []
    temp = []
    count = 0 # 隨機決定index的開頭 
    while len(batch_index) <= 77:  # 1個batch裡只有77筆資料
        temp.append(count)
        count += 1
        if len(temp) == batch_size:
            batch_index.append(temp)
            temp = []
        if count == len(train_source):
            count = 0

    for batch_i in range(0 , 77):
        pad_train_source_batch , \
        pad_train_target_batch , \
        train_source_length , \
        train_target_length = get_batches(source = train_source ,
                                          target = train_target , 
                                          batch_index = batch_index , 
                                          batch_i = batch_i)
        
        _ , loss , predicting_logits_result =\
        sess.run([train_op , cost , predicting_logits], 
                 feed_dict = {input_data : pad_train_source_batch ,
                              targets : pad_train_target_batch ,
                              source_sequence_length: train_source_length , 
                              target_sequence_length : train_target_length  ,            
                              lr: learning_rate})
        
        if batch_i % 30 == 0: # 每隔30個輪查看一下結果
            
            pad_valid_source_batch , \
            pad_valid_target_batch , \
            valid_source_length , \
            valid_target_length = get_batches(source = valid_source ,
                                              target = valid_target , 
                                              batch_index = batch_index , 
                                              batch_i = 0)
            
            validation_loss = sess.run(cost, 
                                       feed_dict = {input_data : pad_valid_source_batch ,
                                                    targets : pad_valid_target_batch ,
                                                    source_sequence_length: valid_source_length , 
                                                    target_sequence_length : valid_target_length})
            
            print('Epoch : {}/{} \nBatch : {}/{} \nTraining Loss : {:.3f} \nValidation loss: {:.3f}'
                  .format(epoch_i , epochs , 
                          batch_i , len(train_source) // batch_size , 
                          loss , validation_loss))
            
            index = np.random.randint(batch_size)
            print('Source : {}'.format([source_int_to_letter[i] for i in pad_train_source_batch[index]] ))
            print('Target : {}'.format([target_int_to_letter[i] for i in pad_train_target_batch[index]] ))
            print('Predict : {}\n'.format([target_int_to_letter[i] for i in predicting_logits_result[index]] ))
    
    
# 保存模型
saver = tf.train.Saver()
saver.save(sess , 'trained_model/save_net')
print('Model Trained and Saved')

Epoch : 0/60 
Batch : 0/77 
Training Loss : 3.400 
Validation loss: 3.393
Source : ['t', 'j', 'q', 'r', 'z', '<PAD>', '<PAD>']
Target : ['j', 'q', 'r', 't', 'z', '<EOS>', '<PAD>', '<PAD>']
Predict : ['q', 'q', 'q', 'q', 'q', 'j', 'j', 'a']

Epoch : 0/60 
Batch : 30/77 
Training Loss : 3.018 
Validation loss: 3.004
Source : ['q', 'o', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['o', 'q', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['<EOS>']

Epoch : 0/60 
Batch : 60/77 
Training Loss : 2.638 
Validation loss: 2.653
Source : ['t', 'w', 'm', 'a', 'm', 'j', '<PAD>']
Target : ['a', 'j', 'm', 'm', 't', 'w', '<EOS>', '<PAD>']
Predict : ['d', 'd', 'e', '<EOS>', '<PAD>']

Epoch : 1/60 
Batch : 0/77 
Training Loss : 2.537 
Validation loss: 2.535
Source : ['n', 'c', 'p', 'q', '<PAD>', '<PAD>', '<PAD>']
Target : ['c', 'n', 'p', 'q', '<EOS>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['b', 'c', 'l', '<EOS>', '<PAD>', '<PAD>']

Epoch : 1/60 
Batch : 30/77 
Training Loss : 2

Epoch : 10/60 
Batch : 30/77 
Training Loss : 0.552 
Validation loss: 0.556
Source : ['q', 'q', 'i', 'r', 'f', 'n', 'q']
Target : ['f', 'i', 'n', 'q', 'q', 'q', 'r', '<EOS>']
Predict : ['i', 'i', 'i', 'q', 'q', 't', 'v', '<EOS>']

Epoch : 10/60 
Batch : 60/77 
Training Loss : 0.507 
Validation loss: 0.524
Source : ['t', 't', 'h', 's', '<PAD>', '<PAD>', '<PAD>']
Target : ['h', 's', 't', 't', '<EOS>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['f', 's', 't', 't', '<EOS>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 11/60 
Batch : 0/77 
Training Loss : 0.506 
Validation loss: 0.501
Source : ['v', 'w', 'c', 'r', 'g', '<PAD>', '<PAD>']
Target : ['c', 'g', 'r', 'v', 'w', '<EOS>', '<PAD>', '<PAD>']
Predict : ['b', 'g', 'p', 'u', 'w', '<EOS>', '<PAD>', '<PAD>']

Epoch : 11/60 
Batch : 30/77 
Training Loss : 0.472 
Validation loss: 0.473
Source : ['i', 'w', 'g', 'y', 'x', 'g', '<PAD>']
Target : ['g', 'g', 'i', 'w', 'x', 'y', '<EOS>', '<PAD>']
Predict : ['g', 'h', 'j', 'v', 'x', 'y', '<EOS>', '<PAD>']

Epoch

Epoch : 21/60 
Batch : 0/77 
Training Loss : 0.142 
Validation loss: 0.145
Source : ['y', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['y', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['y', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 21/60 
Batch : 30/77 
Training Loss : 0.135 
Validation loss: 0.141
Source : ['c', 'a', 'x', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['a', 'c', 'x', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['a', 'c', 'x', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 21/60 
Batch : 60/77 
Training Loss : 0.112 
Validation loss: 0.129
Source : ['u', 'd', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['d', 'u', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['d', 'u', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 22/60 
Batch : 0/77 
Training Loss : 0.128 
Validation loss: 0.132
Source : ['j', 'p', 'j', 'y', 'c', 'f', 'z']
Target :

Epoch : 31/60 
Batch : 60/77 
Training Loss : 0.045 
Validation loss: 0.060
Source : ['p', 'x', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['p', 'x', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['p', 'x', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 32/60 
Batch : 0/77 
Training Loss : 0.056 
Validation loss: 0.064
Source : ['w', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['w', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['w', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 32/60 
Batch : 30/77 
Training Loss : 0.055 
Validation loss: 0.058
Source : ['z', 'g', 'd', 'y', 'q', '<PAD>', '<PAD>']
Target : ['d', 'g', 'q', 'y', 'z', '<EOS>', '<PAD>', '<PAD>']
Predict : ['d', 'g', 'q', 'y', 'z', '<EOS>', '<PAD>', '<PAD>']

Epoch : 32/60 
Batch : 60/77 
Training Loss : 0.042 
Validation loss: 0.056
Source : ['a', 'w', 'k', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['a', 

Epoch : 42/60 
Batch : 0/77 
Training Loss : 0.027 
Validation loss: 0.035
Source : ['u', 'f', 'n', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['f', 'n', 'u', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['f', 'n', 'u', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 42/60 
Batch : 30/77 
Training Loss : 0.028 
Validation loss: 0.034
Source : ['n', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['n', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['n', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 42/60 
Batch : 60/77 
Training Loss : 0.021 
Validation loss: 0.030
Source : ['f', 'x', 'm', 'p', '<PAD>', '<PAD>', '<PAD>']
Target : ['f', 'm', 'p', 'x', '<EOS>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['f', 'm', 'p', 'x', '<EOS>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 43/60 
Batch : 0/77 
Training Loss : 0.026 
Validation loss: 0.034
Source : ['n', 'k', 'g', 's', '<PAD>', '<PAD>', '<PAD>']
Target : ['g', 'k', 

Epoch : 52/60 
Batch : 30/77 
Training Loss : 0.016 
Validation loss: 0.021
Source : ['i', 'm', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['i', 'm', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['i', 'm', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 52/60 
Batch : 60/77 
Training Loss : 0.011 
Validation loss: 0.018
Source : ['p', 'd', 'c', 'g', 'g', '<PAD>', '<PAD>']
Target : ['c', 'd', 'g', 'g', 'p', '<EOS>', '<PAD>', '<PAD>']
Predict : ['c', 'd', 'g', 'g', 'p', '<EOS>', '<PAD>', '<PAD>']

Epoch : 53/60 
Batch : 0/77 
Training Loss : 0.015 
Validation loss: 0.019
Source : ['s', 'd', 'o', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Target : ['d', 'o', 's', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Predict : ['d', 'o', 's', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

Epoch : 53/60 
Batch : 30/77 
Training Loss : 0.015 
Validation loss: 0.019
Source : ['d', 'u', 'z', 'f', '<PAD>', '<PAD>', '<PAD>']
Target : ['d', 'f', 'u', 'z', '<EOS>', '<PA

## Testing

In [14]:
import os    
sess = tf.Session()
new_saver = tf.train.import_meta_graph(os.path.join('trained_model/save_net.meta'))
new_saver.restore(sess, tf.train.latest_checkpoint(os.path.join('trained_model')))

graph = tf.get_default_graph()
input_data = graph.get_tensor_by_name('inputs:0')
targets = graph.get_tensor_by_name('targets:0')
source_sequence_length = graph.get_tensor_by_name('source_sequence_length:0')
target_sequence_length = graph.get_tensor_by_name('target_sequence_length:0')
logits = graph.get_tensor_by_name('predictions:0')

input_word = 'common'

test_source = [] 
for letter in input_word:
    if letter not in source_letter_to_int.keys():
        test_source.append(source_letter_to_int['<UNK>'])
    else:
        test_source.append(source_letter_to_int[letter])
        
test_target = []
for letter in input_word:
    if letter not in target_letter_to_int.keys():
        test_target.append(target_letter_to_int['<UNK>'])
    else:
        test_target.append(target_letter_to_int[letter])

test_target = test_target + [target_letter_to_int['<EOS>']]  

batch_size = 128
test_source = [test_source] * batch_size
test_target = [test_target] * batch_size
test_source_length = [len(i) for i in test_source]
test_target_length = [len(i) for i in test_target]

test_source = np.array(test_source)
test_target = np.array(test_target)
test_source_length = np.array(test_source_length)
test_target_length = np.array(test_target_length)

answer = sess.run(logits , feed_dict = {input_data : test_source ,
                                        targets : test_target ,
                                        source_sequence_length: test_source_length , 
                                        target_sequence_length : test_target_length})
answer = answer[0 , :]
answer_to_letter = []
for num in answer:
    answer_to_letter.append(target_int_to_letter[num])
answer_to_letter       

INFO:tensorflow:Restoring parameters from trained_model\save_net


['c', 'm', 'm', 'n', 'o', 'o', '<EOS>']